In [1]:
import pandas as pd
import sqlite3


In [36]:
orders = pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


In [37]:
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


In [38]:
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

In [39]:
with open("restaurants.sql", "r", encoding="utf-8") as file:
    sql_script = file.read()

cursor.executescript(sql_script)

restaurants = pd.read_sql_query(
    "SELECT * FROM restaurants",
    conn
)

In [40]:
orders_users = pd.merge(
    orders,
    users,
    how="left",
    on="user_id"
)

In [41]:
df = pd.merge(
    orders_users,
    restaurants,
    how="left",
    on="restaurant_id"
)

In [42]:
df.to_csv(
    "final_food_delivery_dataset.csv",
    index=False
)

In [43]:
print(df.shape)
print(df.head())
print(df.isnull().sum())

(10000, 12)
   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                restaurant_name_x       name       city membership  \
0               New Foods Chinese  User_2508  Hyderabad    Regular   
1  Ruchi Curry House Multicuisine  User_2693       Pune    Regular   
2           Spice Kitchen Punjabi  User_2084    Chennai       Gold   
3          Darbar Kitchen Non-Veg   User_319  Bangalore       Gold   
4       Royal Eatery South Indian  User_1064       Pune    Regular   

  restaurant_name_y  cuisine  rating  
0    Restaurant_450  Mexican     3.2  
1    Restaurant_309   Indian     4.5  
2    Restaurant_107  Mexican     4.0  
3    Restaurant_224 

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   order_id           10000 non-null  int64  
 1   user_id            10000 non-null  int64  
 2   restaurant_id      10000 non-null  int64  
 3   order_date         10000 non-null  object 
 4   total_amount       10000 non-null  float64
 5   restaurant_name_x  10000 non-null  object 
 6   name               10000 non-null  object 
 7   city               10000 non-null  object 
 8   membership         10000 non-null  object 
 9   restaurant_name_y  10000 non-null  object 
 10  cuisine            10000 non-null  object 
 11  rating             10000 non-null  float64
dtypes: float64(2), int64(3), object(7)
memory usage: 937.6+ KB


In [45]:
gold_df = df[df['membership'] == 'Gold']
city_rev = gold_df.groupby('city')['total_amount'].sum().reset_index()
top_city_rev = city_rev.loc[city_rev['total_amount'].idxmax()]

print(f"The city with the highest total revenue from Gold members is {top_city_rev['city']}")

The city with the highest total revenue from Gold members is Chennai


In [46]:
cuisine_avg_order = df.groupby('cuisine')['total_amount'].mean().reset_index()
top_cuisine_avg = cuisine_avg_order.loc[cuisine_avg_order['total_amount'].idxmax()]

print(f"The cuisine with the highest average order value is {top_cuisine_avg['cuisine']} with an average of {top_cuisine_avg['total_amount']:.2f}.")

The cuisine with the highest average order value is Mexican with an average of 808.02.


In [47]:
user_orders_sum = df.groupby('user_id')['total_amount'].sum().reset_index()
hv_users = user_orders_sum[user_orders_sum['total_amount'] > 1000]
num_hv_users = hv_users['user_id'].nunique()

print(f"Number of distinct users who placed orders worth more than ₹1000 in total: {num_hv_users}")

Number of distinct users who placed orders worth more than ₹1000 in total: 2544


In [48]:
gold_city_avg_order = gold_df.groupby('city')['total_amount'].mean().reset_index()
top_gold_city_avg = gold_city_avg_order.loc[gold_city_avg_order['total_amount'].idxmax()]

print(f"Among Gold members, the city with the highest average order value is {top_gold_city_avg['city']} with an average order value of {top_gold_city_avg['total_amount']:.2f}.")

Among Gold members, the city with the highest average order value is Chennai with an average order value of 808.46.


In [74]:
distinct_restaurants_per_cuisine = df.groupby('cuisine')['restaurant_id'].nunique().reset_index()
print("Number of distinct restaurants per cuisine type:")
print(distinct_restaurants_per_cuisine)

Number of distinct restaurants per cuisine type:
   cuisine  restaurant_id
0  Chinese            120
1   Indian            126
2  Italian            126
3  Mexican            128


In [75]:
total_revenue_per_cuisine = final_df.groupby('cuisine')['total_amount'].sum().reset_index()
print("Total Revenue per Cuisine Type:")
print(total_revenue_per_cuisine)

Total Revenue per Cuisine Type:
   cuisine  total_amount
0  Chinese    1930504.65
1   Indian    1971412.58
2  Italian    2024203.80
3  Mexican    2085503.09


In [76]:
cuisine_analysis_df = pd.merge(
    distinct_restaurants_per_cuisine,
    total_revenue_per_cuisine,
    on='cuisine'
)
print("Combined Cuisine Analysis DataFrame:")
print(cuisine_analysis_df)

Combined Cuisine Analysis DataFrame:
   cuisine  restaurant_id  total_amount
0  Chinese            120    1930504.65
1   Indian            126    1971412.58
2  Italian            126    2024203.80
3  Mexican            128    2085503.09


In [77]:
avg_total_rev = total_revenue_per_cuisine['total_amount'].mean()
sig_rev_cuisines = cuisine_analysis_df[cuisine_analysis_df['total_amount'] >= avg_total_rev]
target_cuisine = sig_rev_cuisines.loc[sig_rev_cuisines['restaurant_id'].idxmin()]

print(f"The cuisine with the lowest number of distinct restaurants and a significant revenue contribution is {target_cuisine['cuisine']} with {target_cuisine['restaurant_id']} distinct restaurants and a total revenue of {target_cuisine['total_amount']:.2f}.")

The cuisine with the lowest number of distinct restaurants and a significant revenue contribution is Italian with 126 distinct restaurants and a total revenue of 2024203.80.


In [53]:
total_orders = df.shape[0]
print(f"The total number of orders is: {total_orders}")

The total number of orders is: 10000


In [54]:
gold_member_orders = df[df['membership'] == 'Gold'].shape[0]
print(f"The number of orders placed by Gold members is: {gold_member_orders}")

The number of orders placed by Gold members is: 4987


In [56]:
percentage_gold_orders = (gold_member_orders / total_orders) * 100
percentage_gold_orders = round(percentage_gold_orders)
print(f"The percentage of orders placed by Gold members is: {percentage_gold_orders:.2f}%")

The percentage of orders placed by Gold members is: 50.00%


In [79]:
order_count_per_restaurant = df.groupby('restaurant_id').size().reset_index(name='order_count')
print(order_count_per_restaurant.head())

   restaurant_id  order_count
0              1           16
1              2           19
2              3           20
3              4           17
4              5           23


In [78]:
avg_order_value_per_restaurant = df.groupby('restaurant_id')['total_amount'].mean().reset_index()
print(avg_order_value_per_restaurant.head())

   restaurant_id  total_amount
0              1    755.233125
1              2    741.113684
2              3    749.545000
3              4    664.400000
4              5    887.251304


In [80]:
merged_df = pd.merge(
    avg_order_value_per_restaurant,
    order_count_per_restaurant,
    on='restaurant_id'
)
print(merged_df.head())

   restaurant_id  total_amount  order_count
0              1    755.233125           16
1              2    741.113684           19
2              3    749.545000           20
3              4    664.400000           17
4              5    887.251304           23


In [81]:
restaurants_less_than_20_orders = merged_df[merged_df['order_count'] < 20]
print(restaurants_less_than_20_orders.head())

   restaurant_id  total_amount  order_count
0              1    755.233125           16
1              2    741.113684           19
3              4    664.400000           17
5              6    718.878125           16
6              7   1002.140625           16


In [84]:
highest_avg_order_restaurant = restaurants_less_than_20_orders.loc[restaurants_less_than_20_orders['total_amount'].idxmax()]
print(f"The restaurant with the highest average order value among those with less than 20 total orders is Restaurant ID: {highest_avg_order_restaurant['restaurant_id']}, with an average order value of {highest_avg_order_restaurant['total_amount']:.2f}, and {highest_avg_order_restaurant['order_count']} total orders.")

The restaurant with the highest average order value among those with less than 20 total orders is Restaurant ID: 294.0, with an average order value of 1040.22, and 13.0 total orders.


In [88]:
cuisine_city_revenue = df.groupby(['cuisine', 'city'])['total_amount'].sum().reset_index()
print("Revenue per Cuisine and City Combination:")
print(cuisine_city_revenue.head())

Revenue per Cuisine and City Combination:
   cuisine       city  total_amount
0  Chinese  Bangalore     525046.13
1  Chinese    Chennai     491710.42
2  Chinese  Hyderabad     470061.40
3  Chinese       Pune     443686.70
4   Indian  Bangalore     543014.04


In [89]:
highest_revenue_combination = cuisine_city_revenue.loc[cuisine_city_revenue['total_amount'].idxmax()]

print(f"The cuisine and city combination with the highest total revenue is {highest_revenue_combination['cuisine']} in {highest_revenue_combination['city']} with a total revenue of {highest_revenue_combination['total_amount']:.2f}.")

The cuisine and city combination with the highest total revenue is Mexican in Bangalore with a total revenue of 571004.61.


In [90]:
gold_indian_revenue = df[(df['membership'] == 'Gold') & (df['cuisine'] == 'Indian')]['total_amount'].sum()
print(f"Total revenue from Gold members for Indian cuisine: {gold_indian_revenue:.2f}")

Total revenue from Gold members for Indian cuisine: 979312.31


In [91]:
gold_italian_revenue = df[(df['membership'] == 'Gold') & (df['cuisine'] == 'Italian')]['total_amount'].sum()
print(f"Total revenue from Gold members for Italian cuisine: {gold_italian_revenue:.2f}")

Total revenue from Gold members for Italian cuisine: 1005779.05


In [92]:
regular_indian_revenue = df[(df['membership'] == 'Regular') & (df['cuisine'] == 'Indian')]['total_amount'].sum()
print(f"Total revenue from Regular members for Indian cuisine: {regular_indian_revenue:.2f}")

Total revenue from Regular members for Indian cuisine: 992100.27


In [93]:
regular_chinese_revenue = df[(df['membership'] == 'Regular') & (df['cuisine'] == 'Chinese')]['total_amount'].sum()
print(f"Total revenue from Regular members for Chinese cuisine: {regular_chinese_revenue:.2f}")

Total revenue from Regular members for Chinese cuisine: 952790.91


In [94]:
revenue_combinations = {
    'Gold_Indian': gold_indian_revenue,
    'Gold_Italian': gold_italian_revenue,
    'Regular_Indian': regular_indian_revenue,
    'Regular_Chinese': regular_chinese_revenue
}

highest_revenue_label = max(revenue_combinations, key=revenue_combinations.get)
highest_revenue_value = revenue_combinations[highest_revenue_label]

print(f"The combination with the highest revenue is {highest_revenue_label} with a total revenue of {highest_revenue_value:.2f}.")

The combination with the highest revenue is Gold_Italian with a total revenue of 1005779.05.


In [95]:
df['order_date'] = pd.to_datetime(df['order_date'], format='%d-%m-%Y')
print("The 'order_date' column has been converted to datetime objects.")

The 'order_date' column has been converted to datetime objects.


In [96]:
df['quarter'] = df['order_date'].dt.quarter
print("The 'quarter' column has been extracted.")

The 'quarter' column has been extracted.


In [97]:
revenue_by_quarter = df.groupby('quarter')['total_amount'].sum().reset_index()
print("Total revenue per quarter:")
print(revenue_by_quarter)

Total revenue per quarter:
   quarter  total_amount
0        1    2010626.64
1        2    1945348.72
2        3    2037385.10
3        4    2018263.66


In [98]:
highest_revenue_quarter = revenue_by_quarter.loc[revenue_by_quarter['total_amount'].idxmax()]

print(f"The quarter with the highest total revenue is Quarter {highest_revenue_quarter['quarter']} with a total revenue of {highest_revenue_quarter['total_amount']:.2f}.")

The quarter with the highest total revenue is Quarter 3.0 with a total revenue of 2037385.10.


In [99]:
hyderabad_revenue = df[df['city'] == 'Hyderabad']['total_amount'].sum()
rounded_hyderabad_revenue = round(hyderabad_revenue)
print(f"The total revenue generated from orders placed in Hyderabad city (rounded to the nearest integer) is: {rounded_hyderabad_revenue}")

The total revenue generated from orders placed in Hyderabad city (rounded to the nearest integer) is: 1889367


In [100]:
distinct_users_count = df['user_id'].nunique()
print(f"The number of distinct users who placed at least one order is: {distinct_users_count}")

The number of distinct users who placed at least one order is: 2883


In [101]:
avg_order_value_gold_members = gold_df['total_amount'].mean()
print(f"The average order value for Gold members is: {avg_order_value_gold_members:.2f}")

The average order value for Gold members is: 797.15


In [102]:
high_rated_restaurants_orders = df[df['rating'] >= 4.5]
num_orders_high_rated_restaurants = high_rated_restaurants_orders.shape[0]

print(f"The number of orders placed for restaurants with a rating of 4.5 or higher is: {num_orders_high_rated_restaurants}")

The number of orders placed for restaurants with a rating of 4.5 or higher is: 3374


In [103]:
top_gold_city = top_city_rev['city']
orders_in_top_gold_city = df[(df['membership'] == 'Gold') & (df['city'] == top_gold_city)].shape[0]

print(f"The number of orders placed in {top_gold_city} by Gold members is: {orders_in_top_gold_city}")

The number of orders placed in Chennai by Gold members is: 1337
